In [3]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import os

from embrapa_api.preprocessing.preprocessors import BasePreprocessor
from embrapa_api.config import PROJECT_FOLDER

# Producao

In [7]:
try:
    2 + "2"
    rw_producao = (
        pd.read_csv(
            'http://vitibrasil.cnpuv.embrapa.br/download/Producao.csv',
            sep=';'
        )
    )
except Exception as e:
    rw_producao = pd.read_csv(
        os.path.join(PROJECT_FOLDER, 'data/csv_files/producao_vinho/Producao.csv'),
        sep=';'
    )
    print(e) # trocar por logging
    

unsupported operand type(s) for +: 'int' and 'str'


In [59]:
rf_producao = rw_producao.copy()


In [60]:
TIPO_PRODUTO_MAP = {
    "vm": "Vinho de Mesa",
    "vv": "Vinho Fino de Mesa",
    "su": "Suco",
    "de": "Derivados"
}
EXPECTED_AGG_PRODUCTS = set(['DERIVADOS', 'SUCO', 'VINHO DE MESA', 'VINHO FINO DE MESA (VINIFERA)'])


rf_producao = (
    rf_producao
    .melt(
        id_vars=["id", "produto", "control"],
        var_name="ano",
        value_name="producao_L"
    )
    .rename(columns={
        "id": "ID_PRODUTO",
        "produto": "NM_PRODUTO",
        "control": "NM_CONTROLE",
        "ano": "DT_ANO",
        "producao_L": "VR_PRODUCAO_L"
    })
    .astype({
        "ID_PRODUTO": int,
        "NM_PRODUTO": str,
        "NM_CONTROLE": str,
        "DT_ANO": int,
        "VR_PRODUCAO_L": float
    })
    .sort_values(by=["ID_PRODUTO", "DT_ANO"])
)
rf_producao["NM_PRODUTO"] = rf_producao["NM_PRODUTO"].apply(unidecode).str.title()
rf_producao["TIPO_PRODUTO"] = rf_producao["NM_CONTROLE"].str.split("_").str[0].map(TIPO_PRODUTO_MAP)

# os produtos que não possuem tipo de produto são os tipos agregados
assert set(rf_producao.query("TIPO_PRODUTO.isnull()")["NM_CONTROLE"].unique()) == EXPECTED_AGG_PRODUCTS

rf_producao = (
    rf_producao
    .query("TIPO_PRODUTO.notnull()")
    .drop(columns=["NM_CONTROLE"])
)


In [61]:
rw_producao.head()

,id,control,produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,217208604,154264651,146953297,116710345,193875345,177401209,144565438,...,196173123,210308560,86319015,255015187,218375636,144629737,124200414,173899995,195031611,169762429
1,2,vm_Tinto,Tinto,174224052,121133369,118180926,88589019,146544484,144274134,118360170,...,157776363,169811472,75279191,1365957,188270142,121045115,103916391,146075996,162844214,139320884
2,3,vm_Branco,Branco,748400,1160500,1812367,243900,4138768,1441507,1871473,...,37438069,39557250,10727099,217527985,29229970,22032828,19568734,26432799,30198430,27910299
3,4,vm_Rosado,Rosado,42236152,31970782,26960004,27877426,43192093,31685568,24333795,...,958691,939838,312725,36121245,875524,1551794,715289,1391200,1988968,2531246
4,5,VINHO FINO DE MESA (VINIFERA),VINHO FINO DE MESA (VINIFERA),23899346,23586062,21078771,12368410,31644124,39424590,34500590,...,38464314,37148982,18070626,44537870,38707220,37615422,32516686,43474998,47511796,46268556


In [51]:
rf_producao.head(10)

,ID_PRODUTO,NM_PRODUTO,DT_ANO,VR_PRODUCAO_L,TIPO_PRODUTO
1,2,Tinto,1970,174224052.0,Vinho de Mesa
52,2,Tinto,1971,121133369.0,Vinho de Mesa
103,2,Tinto,1972,118180926.0,Vinho de Mesa
154,2,Tinto,1973,88589019.0,Vinho de Mesa
205,2,Tinto,1974,146544484.0,Vinho de Mesa
256,2,Tinto,1975,144274134.0,Vinho de Mesa
307,2,Tinto,1976,118360170.0,Vinho de Mesa
358,2,Tinto,1977,154801826.0,Vinho de Mesa
409,2,Tinto,1978,162917363.0,Vinho de Mesa
460,2,Tinto,1979,154736439.0,Vinho de Mesa


In [52]:
assert (rf_producao.groupby(["ID_PRODUTO"])["NM_PRODUTO"].nunique() == 1).all, "garantir que o nome do produto é único por ID_PRODUTO"

In [53]:
rf_producao.head()

,ID_PRODUTO,NM_PRODUTO,DT_ANO,VR_PRODUCAO_L,TIPO_PRODUTO
1,2,Tinto,1970,174224052.0,Vinho de Mesa
52,2,Tinto,1971,121133369.0,Vinho de Mesa
103,2,Tinto,1972,118180926.0,Vinho de Mesa
154,2,Tinto,1973,88589019.0,Vinho de Mesa
205,2,Tinto,1974,146544484.0,Vinho de Mesa
